# Getting Started with Graphiti and FalkorDB

This notebook demonstrates how to use **Graphiti** with **FalkorDB** to build and query knowledge graphs.

## What is Graphiti?

Graphiti is a developer-friendly framework for building, querying, and managing knowledge graphs using "episodes" — structured or unstructured information pieces like JSON or text. Key features include:

- Automatic entity and relationship extraction
- Semantic and hybrid search capabilities
- Built-in temporal knowledge support
- Advanced graph-based retrieval

## What is FalkorDB?

FalkorDB is a high-performance in-memory graph database queryable with Cypher. It serves as the infrastructure layer while Graphiti acts as the intelligence layer above it.

## Prerequisites

1. **Start FalkorDB** via Docker:
   ```bash
   docker run -p 6379:6379 -p 3000:3000 -it --rm falkordb/falkordb:latest
   ```

2. **Install Graphiti** with FalkorDB support:
   ```bash
   uv pip install graphiti-core[falkordb]
   uv pip install ipykernel
   ```

3. **Access FalkorDB Browser UI** at http://localhost:3000 to visualize your graph data.

## Step 1: Initialize Graphiti with FalkorDB

Connect to your local FalkorDB instance and initialize Graphiti. The `build_indices_and_constraints()` method sets up what Graphiti needs under the hood:

- Entity types and uniqueness constraints
- Search indices for efficient querying
- Schema infrastructure for knowledge graph operations

In [1]:
from graphiti_core import Graphiti
from graphiti_core.driver.falkordb_driver import FalkorDriver

falkor_driver = FalkorDriver(
    host='localhost',
    port=6379,
    username=None,
    password=None,
)
graphiti = Graphiti(graph_driver=falkor_driver)

await graphiti.build_indices_and_constraints()

## Step 2: Add Knowledge via Episodes

Graphiti uses **episodes** to ingest knowledge — these can be unstructured text or structured JSON data. The framework automatically extracts entities and relationships from your data.

In this example, we add two episodes:
1. **Text episode**: An employee spotlight describing Alex Chen joining the team
2. **JSON episode**: Structured HR profile data for Maria Gomez

Key parameters for `add_episode()`:
- `episode_body`: The content (text or JSON string)
- `source`: Episode type (`EpisodeType.text` or `EpisodeType.json`)
- `source_description`: Context about the data source
- `reference_time`: Timestamp for temporal knowledge tracking
- `group_id`: Logical grouping for your knowledge graph

In [2]:
from graphiti_core.nodes import EpisodeType
from datetime import datetime, timezone
import json

episodes = [
            {
                "content": "Alex Chen joined the AI Research team in May 2022 as a Senior ML Engineer reporting to Maria Gomez.",
                "type": EpisodeType.text,
                "description": "employee spotlight",
            },
            {
                "content": {
                    "name": "Maria Gomez",
                    "role": "Head of Product",
                    "start_date": "2019-03-01",
                    "reports_to": "CTO"
                },
                "type": EpisodeType.json,
                "description": "HR profile export",
            },
        ]

for i, ep in enumerate(episodes):
    await graphiti.add_episode(
        name=f"HR Episode {i}",
        episode_body=json.dumps(ep["content"]) if isinstance(ep["content"], dict) else ep["content"],
        source=ep["type"],
        source_description=ep["description"],
        reference_time=datetime.now(timezone.utc),
        group_id="internal-graph"
    )

## Step 3: Query the Knowledge Graph

Use natural language to query your knowledge graph. Behind the scenes, Graphiti performs:

1. **Vector similarity search** on embeddings
2. **BM25 keyword retrieval** for text matching
3. **Result ranking and merging** for optimal answers

The search returns facts extracted from the graph, not just raw data.

In [3]:
results = await graphiti.search("Who is the Head of Product?")
for r in results:
    print(r.fact)

Maria Gomez is Head of Product.
Alex Chen holds the role of Senior ML Engineer.
Maria Gomez reports to the CTO.
Alex Chen joined the AI Research team in May 2022.


## Step 4: Center Node Search (Graph-Aware Reranking)

Improve search relevance by using **center node reranking**. This approach boosts results based on their proximity to a known entity in the graph.

Use cases for center node search:
- Tracing organizational hierarchies
- Following process chains
- Exploring cause-effect relationships

By anchoring the search to a specific node (e.g., Maria Gomez), results are reranked based on graph distance, providing more contextually relevant answers.

In [4]:
center_node_uuid = results[0].source_node_uuid

reranked = await graphiti.search(
    "Who reports to the CTO?",
    center_node_uuid=center_node_uuid
)

In [5]:
print("\nReranked Results:")
for r in reranked:
    print(r.fact)


Reranked Results:
Maria Gomez reports to the CTO.
Maria Gomez is Head of Product.
Alex Chen reports to Maria Gomez.
Alex Chen holds the role of Senior ML Engineer.
Alex Chen joined the AI Research team in May 2022.


## Summary

This notebook demonstrated how Graphiti + FalkorDB simplifies building knowledge graphs:

- **No schema definition required** — Graphiti handles entity/relationship extraction automatically
- **Mixed data types** — Seamlessly ingest both structured JSON and unstructured text
- **Hybrid search** — Combines vector similarity and keyword matching for powerful retrieval
- **Graph-native context** — Center node reranking leverages graph structure for better results

### Next Steps

- Explore the graph visually at http://localhost:3000
- Add more episodes from different data sources
- Experiment with different search queries and center nodes

### Resources

- [FalkorDB Documentation](https://docs.falkordb.com/)
- [Graphiti Documentation](https://github.com/getzep/graphiti)
- [Source Blog Post](https://www.falkordb.com/blog/graphiti-get-started/)